# Домашнее задание 10: Предсказание цен на жильё в Airbnb

В этом задании мы:

- исследуем данные о ценах на жильё в Airbnb в Нью-Йорке;
- проведём предобработку и генерацию признаков;
- построим модели линейной регрессии с регуляризацией и без;
- сравним результаты и сделаем выводы.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler


## 1. Загрузка и первичный осмотр данных

Загружаем датасет Airbnb NYC (если вы работаете локально, скачайте [отсюда](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data)) и делаем первичный анализ.


In [ ]:
# Загрузка датасета (замените путь на свой, если работаете локально)
df = pd.read_csv('/content/AB_NYC_2019.csv')
df.head()


## 2. Очистка данных и удаление ненужных признаков

Удалим следующие признаки, которые не несут полезной информации для модели:
- `id`, `name`, `host_id`, `host_name`, `last_review`


In [ ]:
df.drop(['id', 'name', 'host_id', 'host_name', 'last_review'], axis=1, inplace=True)
df.head()


## 3. Обзор базовых статистик

Посмотрим на базовые статистики по числовым признакам.


In [ ]:
df.drop(['id', 'name', 'host_id', 'host_name', 'last_review'], axis=1, inplace=True)
df.head()


## 3. Обзор базовых статистик

Посмотрим на базовые статистики по числовым признакам.


In [ ]:
df.describe(include='all')


## 4. Обработка пропусков

Проверим наличие пропущенных значений и обработаем их.


In [ ]:
df.isnull().sum()


In [ ]:
# Заменим пропуски в reviews_per_month на 0 (если отзывов нет, то и среднее = 0)
df['reviews_per_month'].fillna(0, inplace=True)

# Удалим оставшиеся строки с пропущенными значениями
df.dropna(inplace=True)


## 5. Визуализация распределений признаков

Построим гистограммы для числовых признаков, сгруппировав по типу комнаты (`room_type`).


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric_columns = df.select_dtypes(include='number').columns

for col in numeric_columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(data=df, x=col, hue='room_type', bins=30, kde=True, palette='tab10')
    plt.title(f'Распределение признака: {col}')
    plt.xlabel(col)
    plt.ylabel('Количество')
    plt.tight_layout()
    plt.show()


## 6. Матрица корреляций

Построим тепловую карту (heatmap) для визуализации корреляций между числовыми признаками.


In [ ]:
plt.figure(figsize=(10, 8))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Матрица корреляций')
plt.show()


## 7. Pairplot для коррелированных признаков

Выберем признаки с высокой корреляцией и построим парные scatterplot-ы.


In [ ]:
# Выберем пары с корреляцией > 0.7, исключая единичную диагональ
high_corr = corr[(corr > 0.7) & (corr < 1.0)].stack().reset_index()
high_corr.columns = ['Feature1', 'Feature2', 'Correlation']
display(high_corr)


In [ ]:
# Построим pairplot для первых 2–3 высоко скоррелированных признаков
selected_features = list(set(high_corr['Feature1'].tolist() + high_corr['Feature2'].tolist()))
sns.pairplot(df[selected_features + ['room_type']], hue='room_type')


## 8. Boxplot по целевой переменной (room_type)

Построим boxplot'ы по различным признакам, сгруппированным по типу комнаты. Это поможет понять, какие признаки лучше всего отделяют категории.


In [ ]:
for col in numeric_columns:
    plt.figure(figsize=(8, 4))
    sns.boxplot(data=df, x='room_type', y=col, palette='pastel')
    plt.title(f'Boxplot: {col} по типу комнаты')
    plt.tight_layout()
    plt.show()


## 9. Выводы по EDA

- `price`, `minimum_nights`, `availability_365` и `number_of_reviews` имеют длинные хвосты — стоит рассмотреть логарифмирование или нормализацию.
- Тип комнаты (`room_type`) сильно влияет на распределение цены — видно, что `Entire home/apt` дороже, чем `Private room`.
- Некоторые признаки имеют сильную корреляцию (например, `longitude` и `latitude`), можно объединить их в новое расстояние до центра.
- Признаки `reviews_per_month` и `number_of_reviews` связаны — возможно, есть смысл оставить только один.
- Визуализация помогает понять, что удалённость от центра и тип жилья — важные факторы, влияющие на цену.


## Часть 2. Feature Engineering и предобработка

В этой части мы:
- удалим ненужные признаки;
- обработаем пропуски;
- создадим новый признак: расстояние до центра Манхэттена;
- закодируем категориальные переменные;
- стандартизируем числовые признаки;
- подготовим данные для обучения моделей.


In [ ]:
# Удалим признаки, которые не пригодятся в моделировании
df = df.drop(columns=['id', 'name', 'host_id', 'host_name', 'last_review'])


In [ ]:
# Заполним пропуски в reviews_per_month нулями
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)


In [ ]:
from math import radians, cos, sin, asin, sqrt

# Функция для расчета расстояния по координатам (в км)
def haversine(lon1, lat1, lon2, lat2):
    # перевод в радианы
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # формула гаверсинусов
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # радиус Земли
    return c * r

# Центр Манхэттена (примерные координаты)
manhattan_center = (-73.9855, 40.7580)

# Добавим признак
df['distance_to_manhattan'] = df.apply(lambda row: haversine(
    row['longitude'], row['latitude'],
    manhattan_center[0], manhattan_center[1]
), axis=1)


### Кодирование категориальных признаков

Для моделей линейной регрессии необходимо преобразовать категориальные переменные в числовые.
Для этого применим one-hot encoding к признакам `neighbourhood_group`, `room_type`.


In [ ]:
# Применим pd.get_dummies
df = pd.get_dummies(df, columns=['neighbourhood_group', 'room_type'], drop_first=True)


### Масштабирование числовых признаков

Модели чувствительны к масштабам переменных, особенно регуляризованные (Ridge, Lasso, ElasticNet).
Поэтому стандартизируем числовые признаки с помощью StandardScaler.


In [ ]:
from sklearn.preprocessing import StandardScaler

# Список числовых признаков (исключая целевую переменную)
numeric_features = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
                    'reviews_per_month', 'calculated_host_listings_count',
                    'availability_365', 'distance_to_manhattan']

scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])


### Разделение данных на обучающую и тестовую выборки

Теперь отделим 30% данных на тестирование и подготовим признаки `X` и целевую переменную `y`.


In [ ]:
from sklearn.model_selection import train_test_split

# Целевая переменная
y = df['price']
X = df.drop(columns=['price'])

# Разделим выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Часть 3. Моделирование

Построим модели линейной регрессии:
- LinearRegression,
- RidgeCV,
- LassoCV,
- ElasticNetCV.

Оценим качество каждой модели с использованием R² и RMSE.


In [ ]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    return rmse, r2


In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_rmse, lr_r2 = evaluate_model(lr, X_test, y_test)

# Ridge Regression
ridge = RidgeCV(alphas=[0.1, 1.0, 10.0])
ridge.fit(X_train, y_train)
ridge_rmse, ridge_r2 = evaluate_model(ridge, X_test, y_test)

# Lasso Regression
lasso = LassoCV(cv=5)
lasso.fit(X_train, y_train)
lasso_rmse, lasso_r2 = evaluate_model(lasso, X_test, y_test)

# ElasticNet
elastic = ElasticNetCV(cv=5)
elastic.fit(X_train, y_train)
elastic_rmse, elastic_r2 = evaluate_model(elastic, X_test, y_test)


### Сравнение моделей по метрикам

Посмотрим, какая модель показала лучшие результаты по RMSE и R².


In [ ]:
results = {
    'LinearRegression': (lr_rmse, lr_r2),
    'RidgeCV': (ridge_rmse, ridge_r2),
    'LassoCV': (lasso_rmse, lasso_r2),
    'ElasticNetCV': (elastic_rmse, elastic_r2)
}

for name, (rmse, r2) in results.items():
    print(f'{name}: RMSE = {rmse:.2f}, R² = {r2:.2f}')


### Визуализация важности признаков

Посмотрим на веса признаков в модели LassoCV, чтобы понять, какие из них наиболее влияют на предсказание.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Создаем DataFrame с коэффициентами
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Lasso Coef': lasso.coef_
}).sort_values(by='Lasso Coef', key=abs, ascending=False)

# Строим график
plt.figure(figsize=(10, 6))
plt.barh(coef_df['Feature'], coef_df['Lasso Coef'])
plt.xlabel('Коэффициенты')
plt.title('Влияние признаков (LassoCV)')
plt.gca().invert_yaxis()
plt.grid(True)
plt.tight_layout()
plt.show()


## Выводы

- Лучшей моделью по метрикам качества оказалась: **(указать вручную по результатам)**.
- Самые значимые признаки по LassoCV: **(вручную отметить несколько)**.
- Добавление новых признаков (например, расстояния до Манхэттена) помогает повысить качество модели.
- Регуляризация Lasso и ElasticNet позволяет отбрасывать неинформативные признаки, что полезно при большом количестве фичей.

Домашнее задание завершено.
